In [1]:
    # %reload_ext XXX.mumble
    ip = get_ipython()
    import CommonMark
    from IPython.core.interactiveshell import InteractiveShell
    from IPython.core.inputsplitter import IPythonInputSplitter
    from IPython.core.compilerop import CachingCompiler
    from IPython.utils.capture import capture_output
    from textwrap import indent
    import ast, json
    from dataclasses import dataclass, field
    from nbconvert.exporters import NotebookExporter, TemplateExporter, MarkdownExporter
    from nbconvert.preprocessors import Preprocessor
    from nbformat import v4, NotebookNode
    from json.scanner import py_make_scanner
    from json.decoder import JSONObject, JSONDecoder, WHITESPACE, WHITESPACE_STR
    from traitlets import Unicode, Any, default, Bool
    from dataclasses import dataclass, field
    from types import ModuleType

In [2]:
    def identity(object, *_, **__): return object

To template:

* Each cell is templated first.

    * That cell is tested for SyntaxErrors
    * A markdown cell will template then extract the source and incrementally built so the lines numbers are correct.

* The source must incrementally template and evaluate the source.
* Each time there is a formatted and valid source the node.Expr should be evaluated within the desired scope.
* There is a stream of lines into a template.  That template needs to be formatted

In [3]:
    from json.decoder import JSONDecoder
    class NBDecoder(JSONDecoder):
        def __init__(self, *, object_hook=None, parse_float=None, 
                     parse_int=None, parse_constant=None, 
                     strict=True, object_pairs_hook=None):
            from json.scanner import py_make_scanner
            super().__init__(object_hook=object_hook, parse_float=parse_float, parse_int=parse_int, 
                              parse_constant=parse_constant, strict=strict, 
                             object_pairs_hook=object_pairs_hook)
            self.parse_object = self.object
            self.scan_once = py_make_scanner(self)
            
        def object(self, s_and_end, strict, scan_once, object_hook, object_pairs_hook, 
                   memo=None, _w=WHITESPACE.match, _ws=WHITESPACE_STR):
            str, id = s_and_end
            object, next = JSONObject(s_and_end, strict, scan_once, object_hook, object_pairs_hook, memo=memo, _w=_w, _ws=_ws)

            if 'cell_type' in object:
                object['metadata'].update({
                    'lineno':  len(str[:next].rsplit('"source":', 1)[0].splitlines())
                })
            if 'source' in object:
                object['source'] = ''.join(object['source'])
                
            return NotebookNode(object), next
        

In [4]:
    class ModuleExporter(TemplateExporter):
        name = Unicode('__main__')
        filename = Unicode('<module exporter>')
        parse = Any()
        format = Bool(True)
        outputs = Bool(True)
        @default('parse')
        def _default_parse(self):
            return get_ipython() and get_ipython().compile.ast_parse or __import__('ast').parse
        def from_filename(ModuleExporter,  filename, resources=None, **dict):
            from pathlib import Path
            ModuleExporter.filename = filename
            ModuleExporter.name = Path(filename).stem
            return super().from_filename(filename, resources, **dict)
        
        def from_file(ModuleExporter,  file_stream, resources=None, **dict):    
            return ModuleExporter.from_notebook_node(v4.new_notebook(
                **json.load(file_stream, cls=NBDecoder)), resources)

        def from_notebook_node(self, nb, resource=None, *, module=None):            
            from textwrap import dedent
            
            if module is None:
                module = ModuleType(self.name)
            module.__doc__ = MarkdownExporter(config={
                'TemplateExporter': {'exclude_output': True}
            }).from_notebook_node(nb)[0]
            with capture_output() as output:
                for cell in nb.cells:
                    if cell['cell_type'] == 'code':    
                        if self.format:
                            source = self.environment.from_string(cell['source']).render(**module.__dict__)
                        try:
                            eval(
                                compile(ast.increment_lineno(self.parse(
                                    source, self.filename, 'exec'
                                ), cell['metadata']['lineno']), self.filename, 'exec'),
                                module.__dict__, module.__dict__)
                        except BaseException as Exception:
                            module.__complete__ = Exception
                            break
                else: module.__complete__ = True

            if self.outputs: module.__output__ = output
                
            return module   

In [11]:
    import ast

In [5]:
    @dataclass
    class Markdown(CachingCompiler): 
        input_transformer_manager: None = None
        repr: callable = field(default=__import__('IPython').display.Markdown)
        parser: CommonMark.Parser = field(default_factory=CommonMark.Parser)
            
        def __post_init__(Markdown): CachingCompiler.__init__(Markdown)
            
        def ast_parse(self, data, filename='<unknown>', symbol='exec'):
            from IPython.display import display
            module, str, Exception = ast.Module(body=[]), """""", None
            
            return self.incremental_module(tuple(
                (node.literal, *node.sourcepos[0]) for node, bool in self.parser.parse(data).walker() 
                if node.t == 'code_block'), filename=filename, symbol=symbol)
        
        def ast_parse_one(self, str, lineno=1, col_offset=0, filename='<unknown>', symbol='exec'):
            module = ast.increment_lineno(super().ast_parse(
                (self.input_transformer_manager and self.input_transformer_manager.transform_cell
                 or identity
                )(
                str), filename=filename, symbol=symbol), lineno-1)
            module.body = [
                setattr(node, 'col_offset', col_offset) or node
                for node in module.body]
            return module
            
        def incremental_module(
            self, lines: list = """""", filename='<unknown>', symbol='exec', *, 
            module: ast.Module = None,
        ) -> ast.Module:
            """Incrementally build a module accounting for magics"""
            if not module: module = ast.Module(body=[])
            str = """"""
            Exception = None
            not_an_expr = lines and lines[0][0].startswith('%%')
            for id, (line, _lineno, _col_offset) in enumerate(lines):
                if not str:
                    lineno, col_offset = _lineno, _col_offset
                str += line
                try:
                    if not not_an_expr or id is len(lines)-1:
                        body = self.ast_parse_one(
                            str, lineno, col_offset, filename, symbol).body
                        
                        if not module.body:
                            if not isinstance(body[0], ast.Expr):
                                not_an_expr = True
                        else:
                            module.body += body
                        if
                        str, Exception = """""", None
                except SyntaxError as BaseException: 
                    Exception = BaseException                
            if Exception: raise Exception

            return ast.fix_missing_locations(module)

                        
    def load_ipython_extension(ip=get_ipython()): 
        ip.compile = Markdown()
        ip.compile.input_transformer_manager = ip.input_transformer_manager
        ip.input_transformer_manager = IPythonInputSplitter(
            logical_line_transforms=[], physical_line_transforms=[], python_line_transforms=[])
        update_hooks(NotebookLoader)
        
    def unload_ipython_extension(ip=get_ipython()): 
        ip.input_transformer_manager = ip.compile.input_transformer_manager
        ip.compile = CachingCompiler()



In [6]:
    from importlib.machinery import SourceFileLoader

    class NotebookLoader(SourceFileLoader):
        EXTENSION_SUFFIXES = '.ipynb',
        def exec_module(Notebook, module): return module.__dict__.update(
            vars(ModuleExporter().from_filename(Notebook.path, module=module)))

In [7]:
    import sys

    def update_hooks(*loaders):
        global _NATIVE_HOOK
        from importlib.machinery import FileFinder

        if loaders:
            for i, hook in enumerate(sys.path_hooks):
                __closure__ = getattr(hook, '__closure__', None)
                if __closure__ and issubclass(__closure__[0].cell_contents, FileFinder):
                    _NATIVE_HOOK = globals().get('_NATIVE_HOOK', (i, hook))
                    sys.path_hooks[i] = FileFinder.path_hook(
                        *_NATIVE_HOOK[1].__closure__[1].cell_contents,
                        *((loader, loader.EXTENSION_SUFFIXES) for loader in loaders
                    ))
        else:
            sys.path_hooks[_NATIVE_HOOK[0]] = _NATIVE_HOOK[1]

        """https://docs.python.org/3/library/sys.html#sys.path_importer_cache"""
        sys.path_importer_cache.clear()

In [8]:
    if __name__ == '__main__': 
        load_ipython_extension()

In [9]:
    if __name__ == '__main__':

Literate markdown cells are tracebacks are consistent
    
        import SomeOutput
        if not SomeOutput.__complete__: raise SomeOutput.__complete__
        print(dir(SomeOutput))
        

['W', '__builtins__', '__call__', '__complete__', '__doc__', '__file__', '__loader__', '__name__', '__output__', '__package__', '__spec__', 'a', 'display', 'x', 'y']


In [10]:
---
    
        SomeOutput.__output__.show()

11
From a template


A Jupyter Widget